In [1]:
import requests
import pandas as pd
import numpy as np
!pip install folium
import folium

     |████████████████████████████████| 102kB 9.6MB/s ta 0:00:011


In [2]:
CLIENT_ID = 'DZPO4SZDH1LBABVXAF2HHXJU3RCG5EG4IADJJ5XSGK13R1AI'
CLIENT_SECRET = 'QFU3YG4HFQHBBCYP0UYKXECWZLFIZQJR4JD0SKWUCQU5MZMM'
VERSION = '20200714'
LIMIT = 40 
print(CLIENT_ID)
print(CLIENT_SECRET)

DZPO4SZDH1LBABVXAF2HHXJU3RCG5EG4IADJJ5XSGK13R1AI
QFU3YG4HFQHBBCYP0UYKXECWZLFIZQJR4JD0SKWUCQU5MZMM


In [6]:
import requests

request_parameters = {
    "client_id" : CLIENT_ID, 
    "client_secret": CLIENT_SECRET, 
    "v": '20200714',
    "section": "grocery",
    "near": "Da Nang",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params = request_parameters)
print("s")

s


In [7]:
#transform data to geocode
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])

In [8]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Da Nang', 'Da Nang')

In [9]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 16.076886011775784, 'lng': 108.22534365611666},
  'sw': {'lat': 16.059918019138294, 'lng': 108.21724196510395}},
 86)

In [11]:
d["geocode"]

{'what': '',
 'where': 'da nang',
 'center': {'lat': 16.06778, 'lng': 108.22083},
 'displayString': 'Da Nang, Thành Phố Đà Nẵng, Vietnam',
 'cc': 'VN',
 'geometry': {'bounds': {'ne': {'lat': 16.11072, 'lng': 108.276871},
   'sw': {'lat': 15.982205, 'lng': 108.141861}}},
 'slug': 'turan-vietnam',
 'longId': '72057594039511928'}

In [12]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [13]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [14]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 50


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5a26a41a31ac6c676705e94c',
  'name': 'cộng cà phê',
  'location': {'address': '39',
   'crossStreet': 'Nguyễn Thái Học',
   'lat': 16.068062879455486,
   'lng': 108.22351222071603,
   'labeledLatLngs': [{'label': 'display',
     'lat': 16.068062879455486,
     'lng': 108.22351222071603}],
   'postalCode': '551105',
   'cc': 'VN',
   'city': 'Đà Nẵng',
   'state': 'Thành Phố Đà Nẵng',
   'country': 'Việt Nam',
   'formattedAddress': ['39 (Nguyễn Thái Học)',
    'Đà Nẵng',
    'Thành Phố Đà Nẵng',
    'Việt Nam']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 're

In [15]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

{'address': '39', 'crossStreet': 'Nguyễn Thái Học', 'lat': 16.068062879455486, 'lng': 108.22351222071603, 'labeledLatLngs': [{'label': 'display', 'lat': 16.068062879455486, 'lng': 108.22351222071603}], 'postalCode': '551105', 'cc': 'VN', 'city': 'Đà Nẵng', 'state': 'Thành Phố Đà Nẵng', 'country': 'Việt Nam', 'formattedAddress': ['39 (Nguyễn Thái Học)', 'Đà Nẵng', 'Thành Phố Đà Nẵng', 'Việt Nam']}
{'address': 'Số 65', 'crossStreet': 'Trần Quốc Toản', 'lat': 16.066321241061253, 'lng': 108.22118053061955, 'labeledLatLngs': [{'label': 'display', 'lat': 16.066321241061253, 'lng': 108.22118053061955}], 'cc': 'VN', 'city': 'Đà Nẵng', 'state': 'Thành Phố Đà Nẵng', 'country': 'Việt Nam', 'formattedAddress': ['Số 65 (Trần Quốc Toản)', 'Đà Nẵng', 'Thành Phố Đà Nẵng', 'Việt Nam']}
{'address': '291 Nguyễn Chí Thanh', 'lat': 16.064596856418454, 'lng': 108.22054089671691, 'labeledLatLngs': [{'label': 'display', 'lat': 16.064596856418454, 'lng': 108.22054089671691}], 'cc': 'VN', 'state': 'Da Nang', 'c

,uid,name,shortname,address,postalcode,lat,lng
0,5a26a41a31ac6c676705e94c,cộng cà phê,Coffee Shop,,551105,16.068063,108.223512
1,546b5fc2498e85d6109593cc,Sanouva Danang Hotel (Khách Sạn Sanouva Đà Nẵng),Hotel,,550000,16.066620,108.220031
2,5aabf13ed552c7495c4b16cc,Thia Go Danang-style restaurant,Vietnamese,,551300,16.076115,108.218690
3,5627c637498e21ca8f40f4be,Leaf Drinks and More,Coffee Shop,,550000,16.067020,108.224790
4,500ab6d4e4b02ee07af15f43,Cơm gà A. Hải,Vietnamese,,59000,16.065276,108.219773


In [16]:
DaNang_center = d["geocode"]["center"]
DaNang_center

{'lat': 16.06778, 'lng': 108.22083}

In [17]:
from folium import plugins


map_DaNang = folium.Map(location=[16.06778, 108.22083], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_DaNang)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_DaNang.add_child(plugins.HeatMap(hm_data))

map_DaNang

In [18]:
lat = 16.06778
lng = 108.22083
map_DaNang = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="O Day Roi!",
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_DaNang)
map_DaNang